In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

# Data

In [2]:
ratings_df = pd.read_csv("ml-25m/ratings.csv")
movies_df = pd.read_csv("ml-25m/movies.csv")

In [3]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [7]:
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [3]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


Remove non-genres entries

In [4]:
nogenres_indexes = movies_df.index[movies_df['genres'] == '(no genres listed)'].tolist()
movieId_todelete = movies_df.iloc[nogenres_indexes]['movieId'].to_numpy()
ratings_df.drop(ratings_df[ratings_df['movieId'].isin(movieId_todelete)].index, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

In [12]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24973468 entries, 0 to 24973467
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.1 MB


In [14]:
ratings_df.sort_values(by="timestamp", inplace=True, ignore_index=True)

In [19]:
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,2262,21,3.0,789652009
1,2262,1079,3.0,789652009
2,2262,47,5.0,789652009
3,109832,34,5.0,822873600
4,109832,32,4.0,822873600


In [21]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24973468 entries, 0 to 24973467
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.1 MB


In [ ]:
ratings_df = ratings_df.to_numpy()

In [29]:
ratings_df.shape

(24973468, 4)

In [32]:
# 676 is a good divisor
batch_size = 676
ratings_df = ratings_df.reshape(ratings_df.shape[0]//batch_size, batch_size, ratings_df.shape[1])

In [33]:
ratings_df.shape

(36943, 676, 4)

36943, [u1, u2, u3, ...] | u1.shape != u2.shape | u1.shape == [n, 4]

36943 == nombre de frames (t = 676) (1 frame == 676 entrées à la suite)
Chercher dans les 676 les utilisateurs et séparer par utilisateur (676 va être split parmis les users ayant votés)



Now we want to create all batches for every single users that we will then feed into the model

In [34]:
ratings_df[0,0,:]

array([2.26200000e+03, 2.10000000e+01, 3.00000000e+00, 7.89652009e+08])

In [ ]:
ratings_df[0,:,0]

In [ ]:
def seperate_drugs():
    new_frames = []
    for frame in frames:
        users_in_frame = []
        

# Model

In [4]:
from utils.convlstmnet import ConvLSTMNet

In [5]:
inputs_channels = 3
cell = "convttlstm"
order = 3
steps = 3
rank = 8
kernel_size = 5
output_sigmoid = True

In [6]:
model = ConvLSTMNet(
        # input to the model
        input_channels = inputs_channels, 
        # architecture of the model
        layers_per_block = (3, 3, 3, 3), 
        hidden_channels = (32, 48, 48, 32), 
        skip_stride = 2,
        # parameters of convolutional tensor-train layers
        cell = cell, cell_params = {"order": order,
        "steps": steps, "rank": rank},
        # parameters of convolutional operations
        kernel_size = kernel_size, bias = True,
        # output function and output format
        output_sigmoid = output_sigmoid)